In [1]:
"""
Run the workflow to get the interferogram and coherence
for the Sentinel-1 data for further processing with 
SNAP to generate INTERFEROMETRIC stacks for STAMPS 
processing.
"""

#%% Import the necessary packages
import asf_search as asf
import numpy as np
import shapely
import matplotlib.pyplot as plt
from copy import deepcopy
from os.path import exists
from os import mkdir

# USe the super master scene thta is just in the middle of the temporal baseline
granule = ['S1A_IW_SLC__1SDV_20190626T155602_20190626T155630_027852_032500_0D6A']
results = asf.granule_search(granule)

for i in np.arange(len(results)):
    print(results[i].properties['fileID'])

reference = results[0]
stack_org = reference.stack()

S1A_IW_SLC__1SDV_20190626T155602_20190626T155630_027852_032500_0D6A-SLC
S1A_IW_SLC__1SDV_20190626T155602_20190626T155630_027852_032500_0D6A-METADATA_SLC


In [2]:
#%% Make a deep copy of the stack
stack = deepcopy(stack_org)

#Loop over stack and remove the None perpendicular or temporal baselines
# and the element of the super master scene with itself where the perpendicular
# and temporal baselines are zeros

frames_nan = []
for i in stack:
    pp_bl = i.properties['perpendicularBaseline']
    tm_pl = i.properties['temporalBaseline']
    if pp_bl == None or tm_pl == None:
        print(i.properties['fileID'], i.properties['temporalBaseline'], 
              i.properties['perpendicularBaseline'])
        frames_nan.append(i)
    # I will comment this part as I will not remove the super master scene
    # if pp_bl == 0 and tm_pl == 0:
    #     print(i.properties['fileID'], i.properties['temporalBaseline'], 
    #           i.properties['perpendicularBaseline'])
    #     frames_nan.append(i)

#Remove the scenes with None/zeros perpendicular or/and temporal baselines
for i in frames_nan:
    stack.remove(i)

S1A_IW_SLC__1SDV_20230617T155626_20230617T155654_049027_05E556_B5CF-SLC 1452 None
S1A_IW_SLC__1SDV_20230629T155626_20230629T155654_049202_05EAA3_0CBB-SLC 1464 None
S1A_IW_SLC__1SDV_20230711T155627_20230711T155655_049377_05F012_5F06-SLC 1476 None
S1A_IW_SLC__1SDV_20230723T155628_20230723T155656_049552_05F565_C6E1-SLC 1488 None
S1A_IW_SLC__1SDV_20230804T155629_20230804T155657_049727_05FACC_DF36-SLC 1500 None
S1A_IW_SLC__1SDV_20230816T155629_20230816T155657_049902_0600AC_191D-SLC 1512 None
S1A_IW_SLC__1SDV_20230828T155630_20230828T155658_050077_0606A8_5873-SLC 1524 None
S1A_IW_SLC__1SDV_20230909T155630_20230909T155658_050252_060C9B_3AAE-SLC 1536 None
S1A_IW_SLC__1SDV_20230921T155631_20230921T155659_050427_06128D_1F35-SLC 1548 None
S1A_IW_SLC__1SDV_20231003T155631_20231003T155659_050602_061896_5AAC-SLC 1560 None
S1A_IW_SLC__1SDV_20231015T155631_20231015T155659_050777_061E99_DD47-SLC 1572 None
S1A_IW_SLC__1SDV_20231027T155631_20231027T155659_050952_062485_9AF9-SLC 1584 None
S1A_IW_SLC__1SDV

In [3]:
#%% Remove the frames that does not have a a specific frame number or relative 
# orbit number to keep only the frames that have the same frame number and 
# relative orbit number
frame = 1173
rel_orb = 130
frames_out = []
for i in stack:
    fr = i.properties['frameNumber']
    rel = i.properties['pathNumber']
    # print(fr, rel)
    if fr != frame or rel != rel_orb:
        print(i.properties['fileID'], i.properties['frameNumber'], 
              i.properties['pathNumber'])
        frames_out.append(i)

for i in frames_out:
    stack.remove(i)


S1A_IW_SLC__1SSV_20141014T155550_20141014T155617_002827_003301_0C9E-SLC 1176 130
S1A_IW_SLC__1SSV_20141014T155525_20141014T155552_002827_003301_78B8-SLC 1171 130
S1A_IW_SLC__1SSV_20141107T155550_20141107T155617_003177_003A87_B237-SLC 1176 130
S1A_IW_SLC__1SSV_20141107T155525_20141107T155552_003177_003A87_7A52-SLC 1171 130
S1A_IW_SLC__1SSV_20141201T155549_20141201T155616_003527_004265_4CA3-SLC 1176 130
S1A_IW_SLC__1SSV_20141201T155524_20141201T155551_003527_004265_C8E5-SLC 1171 130
S1A_IW_SLC__1SDV_20141225T155540_20141225T155608_003877_004A5A_1580-SLC 1174 130
S1A_IW_SLC__1SSV_20150118T155548_20150118T155615_004227_00523B_7C6D-SLC 1176 130
S1A_IW_SLC__1SSV_20150118T155523_20150118T155550_004227_00523B_B169-SLC 1171 130
S1A_IW_SLC__1SSV_20150211T155547_20150211T155614_004577_005A1C_74E5-SLC 1176 130
S1A_IW_SLC__1SSV_20150211T155522_20150211T155549_004577_005A1C_4FE5-SLC 1171 130
S1A_IW_SLC__1SSV_20150307T155547_20150307T155614_004927_006278_EDDB-SLC 1175 130
S1A_IW_SLC__1SSV_20150307T15

In [4]:
#%% Convert the stack to list to prepare it for saving as pickle file
import pickle
stack_lst = [stack]

#Create the data folder if it does not exist
if not exists('./data'):
    mkdir('./data')

#Save the stack as a pickle file
with open('./data/stack.pkl', 'wb') as f:
    pickle.dump(stack_lst, f)